#Belajar Pyspark - pivot dan unpivot

artikel dapat dibaca di www.datalearns247.com

##Mempersiapkan Environment

In [1]:
%pip install pyspark #jalankan untuk google colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=d8561f3369b3c2b792d015bbf53f72ced31d148bb38a896312bc6009a47a6cce
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Import package

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

Create session

In [3]:
spark = SparkSession.builder.appName("Belajar PySpark - Pivot dan Unpivot").getOrCreate()

Mempersiapkan data

In [4]:
data = [['Agus','Fisika','Umum',150],['Windy','Fisika','Khusus',200],
        ['Budi','Biologi','Umum',170],['Dina','Fisika','Khusus',180],
        ['Bayu','Fisika','Umum',160],['Dedi','Biologi','Khusus',185],
        ['Ana','Biologi','Umum',170],['Yani','Biologi','Khusus',175],
        ['Citra','Fisika','Umum',170],['Jaka','Biologi','Khusus',180],
        ['Doni','Biologi','Umum',170],['Hadi','Fisika','Umum',175]]

kolom = ["nama","jurusan","jalur","nilai"]

df = spark.createDataFrame(data,kolom)

df.show()

+-----+-------+------+-----+
| nama|jurusan| jalur|nilai|
+-----+-------+------+-----+
| Agus| Fisika|  Umum|  150|
|Windy| Fisika|Khusus|  200|
| Budi|Biologi|  Umum|  170|
| Dina| Fisika|Khusus|  180|
| Bayu| Fisika|  Umum|  160|
| Dedi|Biologi|Khusus|  185|
|  Ana|Biologi|  Umum|  170|
| Yani|Biologi|Khusus|  175|
|Citra| Fisika|  Umum|  170|
| Jaka|Biologi|Khusus|  180|
| Doni|Biologi|  Umum|  170|
| Hadi| Fisika|  Umum|  175|
+-----+-------+------+-----+



##Grouping dan agregasi biasa

In [5]:
df.groupBy("jalur","jurusan").avg("nilai").show()

+------+-------+----------+
| jalur|jurusan|avg(nilai)|
+------+-------+----------+
|Khusus| Fisika|     190.0|
|Khusus|Biologi|     180.0|
|  Umum|Biologi|     170.0|
|  Umum| Fisika|    163.75|
+------+-------+----------+



##Fungsi `pivot` untuk menampilkan data crosstab

In [ ]:
df.groupBy("jalur") \
  .pivot("jurusan",["Biologi","Fisika"]) \
  .avg("nilai") \
  .show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|  180.0| 190.0|
|  Umum|  170.0|163.75|
+------+-------+------+



###Memanggil fungsi `pivot` tanpa parameter list value

In [6]:
df.groupBy("jalur").pivot("jurusan").avg("nilai").show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|  180.0| 190.0|
|  Umum|  170.0|163.75|
+------+-------+------+



###Menampilkan salah satu value saja

In [20]:
df.groupBy("jalur"). \
  pivot("jurusan",["Biologi"]) \
  .avg("nilai").show()

+------+-------+
| jalur|Biologi|
+------+-------+
|Khusus|  180.0|
|  Umum|  170.0|
+------+-------+



###Menggunakan fungsi agregasi yang lain

In [18]:
df.groupBy("jalur") \
  .pivot("jurusan",["Biologi","Fisika"]) \
  .count() \
  .show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|      3|     2|
|  Umum|      3|     4|
+------+-------+------+



In [21]:
df.groupBy("jalur") \
  .pivot("jurusan",["Biologi","Fisika"]) \
  .min("nilai") \
  .show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|    175|   180|
|  Umum|    170|   150|
+------+-------+------+



###Menggunakan fungsi `agg()` untuk agregasi

In [17]:
df.groupBy("jalur").pivot("jurusan",["Biologi","Fisika"]).agg({"nilai":"min"}).show()
df.groupBy("jalur").pivot("jurusan",["Biologi","Fisika"]).agg(F.expr("min(nilai)")).show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|    175|   180|
|  Umum|    170|   150|
+------+-------+------+

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|    175|   180|
|  Umum|    170|   150|
+------+-------+------+



##Fungsi `unPivot`

In [ ]:
dfPivot = df.groupBy("jalur") \
            .pivot("jurusan",["Biologi","Fisika"]) \
            .avg("nilai")
dfPivot.show()
dfPivot.unpivot(["jalur"], ["Biologi", "Fisika"],
                "jurusan",
                "nilai_rata2").show()

+------+-------+------+
| jalur|Biologi|Fisika|
+------+-------+------+
|Khusus|  180.0| 190.0|
|  Umum|  170.0|163.75|
+------+-------+------+

+------+-------+-----------+
| jalur|jurusan|nilai_rata2|
+------+-------+-----------+
|Khusus|Biologi|      180.0|
|Khusus| Fisika|      190.0|
|  Umum|Biologi|      170.0|
|  Umum| Fisika|     163.75|
+------+-------+-----------+

